# DL Assignment 6
## Sargun Singh (102115078) 4O1D

**Q1** *Use the Iris Dataset to classify the different types of irises. Iris s = dataset is inbuilt in seaborn package.*

In [1]:
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
# Load the Iris dataset
iris = sns.load_dataset("iris")

In [3]:
# Map species names to numerical labels
species_to_label = {species: idx for idx, species in enumerate(iris['species'].unique())}
iris['species'] = iris['species'].map(species_to_label)

In [4]:
# Split features and labels
X = iris.iloc[:, :-1].values  # All columns except the last one
y = iris['species'].values    # Last column

In [5]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [9]:
# Define the Deep Neural Network
class DeepNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(DeepNN, self).__init__()
        self.hidden_layers = nn.ModuleList()
        in_features = input_size
        
        # Add hidden layers
        for hidden_size in hidden_sizes:
            self.hidden_layers.append(nn.Linear(in_features, hidden_size))
            in_features = hidden_size
        
        # Output layer
        self.output_layer = nn.Linear(in_features, output_size)
    
    def forward(self, x):
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        x = self.output_layer(x)  # No activation for the output layer (handled by loss function)
        return x

In [10]:
# Model parameters
input_size = X_train.shape[1]  # Number of features
hidden_sizes = [16, 8]         # Sizes of hidden layers
output_size = 3                # Number of iris species

# Instantiate the model
model = DeepNN(input_size, hidden_sizes, output_size)

In [11]:
# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer

In [12]:
# Train the Model
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print loss for every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 1.0003
Epoch [20/100], Loss: 0.7500
Epoch [30/100], Loss: 0.4281
Epoch [40/100], Loss: 0.2769
Epoch [50/100], Loss: 0.1981
Epoch [60/100], Loss: 0.1281
Epoch [70/100], Loss: 0.0815
Epoch [80/100], Loss: 0.0622
Epoch [90/100], Loss: 0.0522
Epoch [100/100], Loss: 0.0477


In [13]:
# Evaluate the Model
with torch.no_grad():
    # Make predictions
    train_predictions = torch.argmax(model(X_train_tensor), dim=1)
    test_predictions = torch.argmax(model(X_test_tensor), dim=1)
    
    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, train_predictions.numpy())
    test_accuracy = accuracy_score(y_test, test_predictions.numpy())

    print(f'Training Accuracy: {train_accuracy:.4f}')
    print(f'Testing Accuracy: {test_accuracy:.4f}')

Training Accuracy: 0.9833
Testing Accuracy: 1.0000
